In [2]:
# Library Imports

import pandas as pd

In [3]:
# Load datasets
reviews_df = pd.read_csv("Data/reviews_dec18.csv")
reviews_df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'Data/reviews_dec18.csv'

In [1]:
# Dataframe Manipulation

reviews_df = reviews_df.drop_duplicates(subset='comments', keep='first')
reviews_df = reviews_df.drop('date', axis=1)
reviews_df = reviews_df.drop('reviewer_id', axis=1)
reviews_df = reviews_df.drop('reviewer_name', axis=1)
reviews_df['comments'] = reviews_df['comments'].astype(str)
reviews_df.info()

NameError: name 'reviews_df' is not defined

In [67]:
# Establish Positive and Negative Cleanliness keywords
pos_keywords = ['clean', 'tidy', 'immaculate', 'sanitary', 'spotless', 'neat'] # Reasoning: https://www.thesaurus.com/browse/clean
neg_keywords = ['dirty', 'dusty', 'filthy', 'messy', 'stained', 'unkempt'] # Reasoning: https://www.thesaurus.com/browse/dirty

In [68]:
# Designating functions

# Identifies keywords
def keyword_filtered_reviews(reviews_df, pos_keywords, neg_keywords):
    # Check Comment is String 
    reviews_df['comments'] = reviews_df['comments'].astype(str)
    
    # Check if keywords in text
    def keyword_in_text(keywords, text):
        for word in keywords:
            if word in text:
                return True
        return False
    
    # Create masks for positive and negative comments
    pos_mask = reviews_df['comments'].apply(lambda x: keyword_in_text(pos_keywords, x.lower()))
    neg_mask = reviews_df['comments'].apply(lambda x: keyword_in_text(neg_keywords, x.lower()))
    
    # Combine masks
    mask = pos_mask | neg_mask
    
    # Filter dataframe by mask
    return f_r_df[mask]

#Counts Keywords
def count_keywords_in_reviews(reviews_df, pos_keywords, neg_keywords):
    # Check Comment is String
    reviews_df['comments'] = reviews_df['comments'].astype(str)
    
    # Count keywords in text
    def keywords_count(keywords, text):
        count = 0
        for word in keywords:
            count += text.lower().count(word)
        return count
    
    #Count Both pos and neg keywords
    reviews_df['pos_keyword_count'] = reviews_df['comments'].apply(lambda x: keywords_count(pos_keywords, x))
    reviews_df['neg_keyword_count'] = reviews_df['comments'].apply(lambda x: keywords_count(neg_keywords, x))
    
    return f_r_df

In [69]:
f_r_df = keyword_filtered_reviews(reviews_df, pos_keywords, neg_keywords)

f_r_df = count_keywords_in_reviews(f_r_df, pos_keywords, neg_keywords)

f_r_df

C:\Users\zhard\AppData\Local\Temp\ipykernel_12636\3798907570.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return f_r_df[mask]


,listing_id,id,comments,pos_keyword_count,neg_keyword_count
0,12351,68394,This was my first Airbnb experience and I am h...,1,0
11,12351,141277,Upon my arrival I was warmly greeted with a ho...,2,0
20,12351,180715,Vinh and Stu are awesome hosts! Even their dog...,1,0
29,12351,210037,Vinh and Stuart are really friendly and helpfu...,1,0
33,12351,225253,Staying at Stuart and Vihn's home was my first...,1,0
...,...,...,...,...,...
446674,30412858,355808013,If you're heading to Sydney and hoping to stay...,1,0
446682,30429699,355602963,A must book place if you are looking for neat ...,2,0
446688,30432487,355608144,"The apartment was as described, clean and mode...",1,0
446696,30489756,355617016,We absolutely loved the place. Apartment was s...,1,0


In [70]:
# Group listing_id & keyword counts sum
k_sum_df = f_r_df.groupby('listing_id')[['pos_keyword_count', 'neg_keyword_count']].sum().reset_index()

k_sum_df

,listing_id,pos_keyword_count,neg_keyword_count
0,12351,114,0
1,14250,2,0
2,15253,75,2
3,20865,4,0
4,26174,16,1
...,...,...,...
16583,30412858,2,0
16584,30429699,2,0
16585,30432487,1,0
16586,30489756,1,0


In [71]:
#Percentage generation

# Make totals
k_sum_df['total_keywords'] = k_sum_df['pos_keyword_count'] + k_sum_df['neg_keyword_count']

# Make Percentage
k_sum_df['pos_keyword_percentage'] = (k_sum_df['pos_keyword_count'] / k_sum_df['total_keywords']) * 100

In [72]:
# table cleaning

#Drop attribute
k_sum_df = k_sum_df.drop('total_keywords', axis=1)

#Order by decending 'pos_keyword_count'
k_sum_df = k_sum_df.sort_values(by='pos_keyword_count', ascending=False)

k_sum_df

,listing_id,pos_keyword_count,neg_keyword_count,pos_keyword_percentage
5531,12954762,243,1,99.590164
5675,13279754,218,0,100.000000
3218,7944819,198,0,100.000000
116,389876,172,0,100.000000
386,952331,170,0,100.000000
...,...,...,...,...
16328,29635366,0,1,0.000000
11804,22089381,0,2,0.000000
15776,28450711,0,1,0.000000
13037,23402460,0,1,0.000000


In [74]:
# Export filtered data
k_sum_df.to_csv('fitered_Cleanliness_data.csv', index=False)